In [1]:
import os
import pandas as pd
import sqlite3
from pathlib import Path
from tqdm.auto import tqdm

# Specify input directory
input_dir = "/kaggle/input/website-framework-data-initial"

# List all files in the input directory
input_files = sorted(os.listdir(input_dir))

TABLE_NAME = "CrUXData"

# Initialize an SQLite database
db_path = "/kaggle/working/crux_data.db"
conn = sqlite3.connect(db_path)

# Process each .csv.gzip file
for filename in tqdm(input_files):
    if filename.endswith(".csv.gzip"):
        input_path = os.path.join(input_dir, filename)

        # Read the .csv.gzip file
        df = pd.read_csv(input_path, compression="gzip")
        
        # Preprocessing: split origin into "domain name" and "protocol"
        df[['protocol', 'domain']] = df['origin'].str.split('//', n=1, expand=True)
        del df['origin']
        
        # Save the DataFrame to the SQLite database
        df.to_sql(TABLE_NAME, conn, if_exists="append", index=False)
        
# vacuum the database to save disk space
cursor = conn.cursor()
cursor.execute(f"VACUUM")
cursor.close()

# Close the database connection
conn.close()

print(f"All data saved to {db_path}")

  0%|          | 0/170 [00:00<?, ?it/s]

All data saved to /kaggle/working/crux_data.db
